# Numerical Methods for Solving Ordinary Differential Equations (ODEs)

Numerical methods for ODEs are essential when analytical solutions are difficult or impossible to obtain. Here are the most important methods with algorithms and Python implementations.

## 1. Initial Value Problems (IVPs)

### Euler's Method (Explicit)
**Algorithm**:
```
Given dy/dt = f(t,y), y(t₀) = y₀, step size h
For n = 0 to N-1:
    yₙ₊₁ = yₙ + h·f(tₙ, yₙ)
    tₙ₊₁ = tₙ + h
```

**Python Implementation**:

In [30]:
import numpy as np
def euler(f, t_span, y0, h):
    t0, tf = t_span
    n = int((tf - t0)/h)
    t = np.linspace(t0, tf, n+1)
    y = np.zeros(n+1)
    y[0] = y0
    
    for i in range(n):
        y[i+1] = y[i] + h * f(t[i], y[i])
    
    return t, y

# Example: y' = -2y, y(0) = 1
f = lambda t, y: -2*y
t, y = euler(f, [0, 2], 1, 0.1)
print (t, y)

[0.  0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1.  1.1 1.2 1.3 1.4 1.5 1.6 1.7
 1.8 1.9 2. ] [1.         0.8        0.64       0.512      0.4096     0.32768
 0.262144   0.2097152  0.16777216 0.13421773 0.10737418 0.08589935
 0.06871948 0.05497558 0.04398047 0.03518437 0.0281475  0.022518
 0.0180144  0.01441152 0.01152922]


### Runge-Kutta Methods

#### RK4 (4th Order)
**Algorithm**:
```
Given dy/dt = f(t,y), y(t₀) = y₀, step size h
For n = 0 to N-1:
    k₁ = h·f(tₙ, yₙ)
    k₂ = h·f(tₙ + h/2, yₙ + k₁/2)
    k₃ = h·f(tₙ + h/2, yₙ + k₂/2)
    k₄ = h·f(tₙ + h, yₙ + k₃)
    yₙ₊₁ = yₙ + (k₁ + 2k₂ + 2k₃ + k₄)/6
    tₙ₊₁ = tₙ + h
```

**Python Implementation**:

In [33]:
def rk4(f, t_span, y0, h):
    t0, tf = t_span
    n = int((tf - t0)/h)
    t = np.linspace(t0, tf, n+1)
    y = np.zeros(n+1)
    y[0] = y0
    
    for i in range(n):
        k1 = h * f(t[i], y[i])
        k2 = h * f(t[i] + h/2, y[i] + k1/2)
        k3 = h * f(t[i] + h/2, y[i] + k2/2)
        k4 = h * f(t[i] + h, y[i] + k3)
        y[i+1] = y[i] + (k1 + 2*k2 + 2*k3 + k4)/6
    
    return t, y

## 2. Adaptive Step Size Methods

### Runge-Kutta-Fehlberg (RKF45)
**Algorithm**:
```
Uses two RK methods (4th and 5th order) to estimate error
Adjust step size based on error tolerance
```

**Python Implementation**:

In [36]:
def rkf45(f, t_span, y0, tol=1e-6, h_max=0.1, h_min=0.01):
    t0, tf = t_span
    t = [t0]
    y = [y0]
    h = h_max
    
    while t[-1] < tf:
        if t[-1] + h > tf:
            h = tf - t[-1]
            
        # Calculate both RK4 and RK5
        k1 = h * f(t[-1], y[-1])
        k2 = h * f(t[-1] + h/4, y[-1] + k1/4)
        k3 = h * f(t[-1] + 3*h/8, y[-1] + 3*k1/32 + 9*k2/32)
        k4 = h * f(t[-1] + 12*h/13, y[-1] + 1932*k1/2197 - 7200*k2/2197 + 7296*k3/2197)
        k5 = h * f(t[-1] + h, y[-1] + 439*k1/216 - 8*k2 + 3680*k3/513 - 845*k4/4104)
        k6 = h * f(t[-1] + h/2, y[-1] - 8*k1/27 + 2*k2 - 3544*k3/2565 + 1859*k4/4104 - 11*k5/40)
        
        # Error estimate
        y4 = y[-1] + 25*k1/216 + 1408*k3/2565 + 2197*k4/4104 - k5/5
        y5 = y[-1] + 16*k1/135 + 6656*k3/12825 + 28561*k4/56430 - 9*k5/50 + 2*k6/55
        error = np.abs(y5 - y4)
        
        # Step size adjustment
        if error < tol:
            t.append(t[-1] + h)
            y.append(y4)
            h = min(h_max, 0.9 * h * (tol/error)**0.2)
        else:
            h = max(h_min, 0.9 * h * (tol/error)**0.25)
    
    return np.array(t), np.array(y)

## 3. Systems of ODEs

### Vectorized RK4 for Systems

In [39]:
def rk4_system(f, t_span, y0, h):
    t0, tf = t_span
    n = int((tf - t0)/h)
    t = np.linspace(t0, tf, n+1)
    y = np.zeros((n+1, len(y0)))
    y[0] = y0
    
    for i in range(n):
        k1 = h * f(t[i], y[i])
        k2 = h * f(t[i] + h/2, y[i] + k1/2)
        k3 = h * f(t[i] + h/2, y[i] + k2/2)
        k4 = h * f(t[i] + h, y[i] + k3)
        y[i+1] = y[i] + (k1 + 2*k2 + 2*k3 + k4)/6
    
    return t, y

# Example: Lorenz system
def lorenz(t, y, sigma=10, beta=8/3, rho=28):
    return np.array([
        sigma*(y[1] - y[0]),
        y[0]*(rho - y[2]) - y[1],
        y[0]*y[1] - beta*y[2]
    ])

t, y = rk4_system(lambda t, y: lorenz(t, y), [0, 50], [1, 1, 1], 0.01)

## 4. Boundary Value Problems (BVPs)

### Shooting Method
**Algorithm**:
```
1. Convert BVP to IVP with initial guess for unknown boundary conditions
2. Solve IVP numerically
3. Compare solution at boundary with target value
4. Adjust guess using root-finding (e.g., secant method)
```

**Python Implementation**:

In [42]:
from scipy.optimize import root_scalar

def shooting_method(f, t_span, bc, guess_range, tol=1e-6):
    """Solve y'' = f(x,y,y') with boundary conditions"""
    a, b = t_span
    ya, yb = bc
    
    def objective(s):
        # Solve IVP with y(a)=ya, y'(a)=s
        def ivp_system(x, z):
            return np.array([z[1], f(x, z[0], z[1])])
        
        _, sol = rk4_system(ivp_system, [a, b], [ya, s], 0.01)
        return sol[-1, 0] - yb  # Difference from target
    
    # Find correct initial slope
    result = root_scalar(objective, bracket=guess_range, method='brentq')
    s_star = result.root
    
    # Final solution with optimal slope
    def final_system(x, z):
        return np.array([z[1], f(x, z[0], z[1])])
    
    return rk4_system(final_system, [a, b], [ya, s_star], 0.01)

# Example: y'' = -y, y(0)=0, y(π/2)=1
f = lambda x, y, dy: -y
t, y = shooting_method(f, [0, np.pi/2], [0, 1], [0, 2])

## 5. Stiff Equations

### Implicit Euler Method
**Algorithm**:
```
For stiff equations where explicit methods fail
yₙ₊₁ = yₙ + h·f(tₙ₊₁, yₙ₊₁)
Requires solving nonlinear equation at each step
```

**Python Implementation**:

In [45]:
from scipy.optimize import fsolve

def implicit_euler(f, t_span, y0, h):
    t0, tf = t_span
    n = int((tf - t0)/h)
    t = np.linspace(t0, tf, n+1)
    y = np.zeros(n+1)
    y[0] = y0
    
    for i in range(n):
        # Solve y[i+1] - y[i] - h*f(t[i+1], y[i+1]) = 0
        func = lambda y_next: y_next - y[i] - h*f(t[i+1], y_next)
        y[i+1] = fsolve(func, y[i])[0]
    
    return t, y

# Example: Stiff equation y' = -1000(y - sin(t)) + cos(t)
f = lambda t, y: -1000*(y - np.sin(t)) + np.cos(t)
t, y = implicit_euler(f, [0, 1], 0, 0.01)

These methods provide a comprehensive toolkit for solving various types of ODEs numerically. The choice of method depends on:

    Problem stiffness
    Desired accuracy
    Computational efficiency requirements
    Whether it's an IVP or BVP

For production use, consider specialized libraries like scipy.integrate.solve_ivp which implements many of these methods with additional optimizations.

## **scipy.integrate.solve_ivp(fun, t_span, y0)**   for initial value problems

## **scipy.integrate.solve_bvp** for boundary value problems

## **FEniCS** or **Firedrake** for finite element methods

# Solving Higher-Order Differential Equations: Numerical Methods

Higher-order differential equations (ODEs) can be transformed into systems of first-order equations and solved using numerical techniques. Here's a comprehensive guide to solving them:

## 1. Converting to First-Order Systems

Any nth-order ODE can be converted to a system of n first-order ODEs:

**Example**: For a 2nd-order ODE:
```
y'' + p(x)y' + q(x)y = f(x)
```
Define:
```
y₁ = y
y₂ = y'
```
Then the system becomes:
```
y₁' = y₂
y₂' = f(x) - p(x)y₂ - q(x)y₁
```

## 2. Numerical Solution Methods

### Runge-Kutta 4th Order (RK4) for Higher-Order ODEs

**Algorithm**:
1. Convert to first-order system
2. Apply RK4 to each equation simultaneously

**Python Implementation**:
```python
import numpy as np

def solve_2nd_order(f, p, q, x_span, y0, dy0, h):
    """Solves y'' + p(x)y' + q(x)y = f(x)"""
    
    def system(x, z):
        return np.array([z[1], f(x) - p(x)*z[1] - q(x)*z[0]])
    
    x0, xf = x_span
    n = int((xf - x0)/h)
    x = np.linspace(x0, xf, n+1)
    z = np.zeros((n+1, 2))
    z[0] = [y0, dy0]
    
    for i in range(n):
        k1 = h * system(x[i], z[i])
        k2 = h * system(x[i] + h/2, z[i] + k1/2)
        k3 = h * system(x[i] + h/2, z[i] + k2/2)
        k4 = h * system(x[i] + h, z[i] + k3)
        z[i+1] = z[i] + (k1 + 2*k2 + 2*k3 + k4)/6
    
    return x, z[:,0]  # Return x and y(x)

# Example: y'' + y = 0, y(0)=1, y'(0)=0 (solution: y=cos(x))
x, y = solve_2nd_order(lambda x: 0, lambda x: 0, lambda x: 1, 
                       [0, 10], 1, 0, 0.1)
```

## 3. Special Methods for Stiff Higher-Order ODEs

### Newmark-β Method (for 2nd-order ODEs)

**Algorithm**:
```
For structural dynamics problems:
M y'' + C y' + K y = F(t)
```

**Python Implementation**:
```python
def newmark_beta(M, C, K, F, t_span, y0, dy0, dt, beta=0.25, gamma=0.5):
    t0, tf = t_span
    n = int((tf - t0)/dt)
    t = np.linspace(t0, tf, n+1)
    y = np.zeros(n+1)
    dy = np.zeros(n+1)
    ddy = np.zeros(n+1)
    
    y[0] = y0
    dy[0] = dy0
    ddy[0] = (F(t[0]) - C*dy[0] - K*y[0])/M
    
    a0 = 1/(beta*dt**2)
    a1 = gamma/(beta*dt)
    a2 = 1/(beta*dt)
    a3 = 1/(2*beta) - 1
    a4 = gamma/beta - 1
    a5 = dt/2*(gamma/beta - 2)
    a6 = dt*(1 - gamma)
    a7 = gamma*dt
    
    K_hat = K + a0*M + a1*C
    
    for i in range(n):
        F_hat = F(t[i+1]) + M*(a0*y[i] + a2*dy[i] + a3*ddy[i]) + C*(a1*y[i] + a4*dy[i] + a5*ddy[i])
        y[i+1] = F_hat/K_hat
        ddy[i+1] = a0*(y[i+1] - y[i]) - a2*dy[i] - a3*ddy[i]
        dy[i+1] = dy[i] + a6*ddy[i] + a7*ddy[i+1]
    
    return t, y
```

## 4. Boundary Value Problems (BVPs) for Higher-Order ODEs

### Finite Difference Method

**Example for 4th-order ODE**:
```python
def solve_4th_order_bvp(f, a, b, ya, yb, yaa, ybb, n):
    h = (b - a)/n
    x = np.linspace(a, b, n+1)
    
    # Create coefficient matrix
    A = np.zeros((n+1, n+1))
    A[0, 0] = 1
    A[-1, -1] = 1
    
    for i in range(1, n):
        A[i, i-2] = 1
        A[i, i-1] = -4
        A[i, i] = 6
        A[i, i+1] = -4
        if i+2 <= n:
            A[i, i+2] = 1
    
    # Create right-hand side
    b = np.zeros(n+1)
    b[0] = ya
    b[-1] = yb
    b[1] += yaa*h**2
    b[-2] += ybb*h**2
    
    for i in range(2, n-1):
        b[i] = f(x[i])*h**4
    
    # Solve system
    y = np.linalg.solve(A, b)
    return x, y
```

## 5. Advanced Methods

### Spectral Methods (Using Chebyshev Polynomials)

```python
def spectral_method_solve(n, L, f, bc):
    # Create Chebyshev differentiation matrix
    x = np.cos(np.pi*np.arange(n+1)/n)
    D = np.zeros((n+1, n+1))
    
    c = np.ones(n+1)
    c[0] = 2
    c[-1] = 2
    
    for i in range(n+1):
        for j in range(n+1):
            if i != j:
                D[i,j] = ((-1)**(i+j)*c[i])/(c[j]*(x[i]-x[j])))
            elif i == 0:
                D[i,j] = (2*(n)**2 + 1)/6
            elif i == n:
                D[i,j] = -(2*(n)**2 + 1)/6
            else:
                D[i,j] = -x[i]/(2*(1-x[i]**2))
    
    # Adjust for domain [0,L]
    x = L*(x + 1)/2
    D = 2/L * D
    
    # Build higher-order derivatives
    D2 = D @ D
    D3 = D @ D2
    D4 = D2 @ D2
    
    # Apply boundary conditions
    A = D4[1:-1, 1:-1]  # For 4th-order problem
    b = f(x[1:-1]) - D4[1:-1,0]*bc[0] - D4[1:-1,-1]*bc[1]
    
    # Solve
    y_interior = np.linalg.solve(A, b)
    y = np.zeros(n+1)
    y[0] = bc[0]
    y[-1] = bc[1]
    y[1:-1] = y_interior
    
    return x, y
```

## Key Considerations:

1. **Order Reduction**: Always convert higher-order ODEs to first-order systems
2. **Stiffness**: For stiff problems, use implicit methods (Backward Differentiation Formulas)
3. **Boundary Conditions**: BVPs require different approaches (shooting, finite difference, spectral)
4. **Accuracy**: Higher-order methods (RK4, spectral) generally provide better accuracy
5. **Stability**: Implicit methods are more stable for stiff equations

For production use, consider specialized libraries like:
- `scipy.integrate.solve_ivp` for initial value problems
- `scipy.integrate.solve_bvp` for boundary value problems
- `FEniCS` or `Firedrake` for finite element methods



In [67]:
from scipy import *

In [71]:
help (integrate.solve_bvp)

Help on function solve_bvp in module scipy.integrate._bvp:

solve_bvp(fun, bc, x, y, p=None, S=None, fun_jac=None, bc_jac=None, tol=0.001, max_nodes=1000, verbose=0, bc_tol=None)
    Solve a boundary value problem for a system of ODEs.

    This function numerically solves a first order system of ODEs subject to
    two-point boundary conditions::

        dy / dx = f(x, y, p) + S * y / (x - a), a <= x <= b
        bc(y(a), y(b), p) = 0

    Here x is a 1-D independent variable, y(x) is an n-D
    vector-valued function and p is a k-D vector of unknown
    parameters which is to be found along with y(x). For the problem to be
    determined, there must be n + k boundary conditions, i.e., bc must be an
    (n + k)-D function.

    The last singular term on the right-hand side of the system is optional.
    It is defined by an n-by-n matrix S, such that the solution must satisfy
    S y(a) = 0. This condition will be forced during iterations, so it must not
    contradict boundary condit

In [75]:
help (integrate.solve_ivp )

Help on function solve_ivp in module scipy.integrate._ivp.ivp:

solve_ivp(fun, t_span, y0, method='RK45', t_eval=None, dense_output=False, events=None, vectorized=False, args=None, **options)
    Solve an initial value problem for a system of ODEs.

    This function numerically integrates a system of ordinary differential
    equations given an initial value::

        dy / dt = f(t, y)
        y(t0) = y0

    Here t is a 1-D independent variable (time), y(t) is an
    N-D vector-valued function (state), and an N-D
    vector-valued function f(t, y) determines the differential equations.
    The goal is to find y(t) approximately satisfying the differential
    equations, given an initial value y(t0)=y0.

    Some of the solvers support integration in the complex domain, but note
    that for stiff ODE solvers, the right-hand side must be
    complex-differentiable (satisfy Cauchy-Riemann equations [11]_).
    To solve a problem in the complex domain, pass y0 with a complex data type.

In [81]:
from scipy import *

In [83]:
help (Firedrake)

NameError: name 'Firedrake' is not defined